In [1]:
%%capture
!pip install feature-engine
!pip install optuna

In [2]:
import gc
import glob
import os
import sys
import time
import traceback
from contextlib import contextmanager
from enum import Enum
from typing import List, Optional
import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel, delayed
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import minmax_scale
from sklearn import preprocessing
from tqdm import tqdm

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader,Dataset
from torch.autograd import Variable
import torch.optim as optim
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
from sklearn.model_selection import train_test_split

import random
import xgboost as xg
import pickle
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder

import datetime

import warnings
warnings.simplefilter("ignore")

In [3]:
train_raw = pd.read_csv('../wids_datathon2023/train_data.csv', parse_dates=["startdate"])
test_raw = pd.read_csv('../wids_datathon2023/test_data.csv', parse_dates=["startdate"])
submit = pd.read_csv('../wids_datathon2023/sample_solution.csv')
target = 'contest-tmp2m-14d__tmp2m'
TARGET = target

In [4]:
def rmse(actual, predicted):
    return mean_squared_error(actual, predicted, squared=False)

def location_nom(train, test):
    scale = 14

    train.loc[:,'lat']=round(train.lat,scale)
    train.loc[:,'lon']=round(train.lon,scale)
    test.loc[:,'lat']=round(test.lat,scale)
    test.loc[:,'lon']=round(test.lon,scale)

    all_df = pd.concat([train, test], axis=0)
    all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()
    train = all_df.iloc[:len(train)]
    test = all_df.iloc[len(train):].drop(target, axis=1)
    
    return train, test

def categorical_encode(train, test):
    le = LabelEncoder()
    train['climateregions__climateregion'] = le.fit_transform(train['climateregions__climateregion'])
    test['climateregions__climateregion'] = le.transform(test['climateregions__climateregion'])
    return train, test

def one_hot_categorical_encode(train, test):
    ohe_enc = fe_OneHotEncoder(
    top_categories=None,
    variables=['climateregions__climateregion'],  # we can select which variables to encode
    drop_last=True)  # to return k-1, false to return k
    
    ohe_enc.fit(train)
    train = ohe_enc.transform(train)
    test = ohe_enc.transform(test)
    return train, test
    
def creat_new_featute(df):
    df['year'] = df['startdate'].dt.year
    df['month'] = df['startdate'].dt.month
    df['day_of_year'] = df['startdate'].dt.dayofyear
    return df

def filter_na_cols(df):
    count_na_df = df.isna().sum() 
    if count_na_df[count_na_df > 0].tolist():
        return count_na_df[count_na_df > 0]
    else:
        return 'Clean dataset'


def feature_engineering_tabnet(train_raw, test_raw):
    train, test = location_nom(train_raw, test_raw)
    train = creat_new_featute(train)
    test = creat_new_featute(test)
    drop_cols = ['index', 'startdate', 'lat', 'lon', target]
    features = [col for col in train.columns if col not in drop_cols]
    X = train[features]
    X_test = test[features]
    y = train[target]
    

    return X, y, X_test

In [5]:
X, y, X_test = feature_engineering_tabnet(train_raw.copy(), test_raw.copy())

In [6]:
col_na = [
    'nmme0-tmp2m-34w__ccsm30', 
    'nmme-tmp2m-56w__ccsm3', 
    'nmme-prate-34w__ccsm3', 
    'nmme0-prate-56w__ccsm30', 
    'nmme0-prate-34w__ccsm30', 
    'nmme-prate-56w__ccsm3', 
    'nmme-tmp2m-34w__ccsm3']

g_means =  ['nmme0-tmp2m-34w__nmme0mean', 
 'nmme-tmp2m-56w__nmmemean', 
 'nmme-prate-34w__nmmemean', 
 'nmme0-prate-56w__nmme0mean', 
 'nmme0-prate-34w__nmme0mean', 
 'nmme-prate-56w__nmmemean', 
 'nmme-tmp2m-34w__nmmemean']


g_1 = ['nmme0-tmp2m-34w__cancm30',
'nmme0-tmp2m-34w__cancm40',
'nmme0-tmp2m-34w__ccsm40',
'nmme0-tmp2m-34w__cfsv20',
'nmme0-tmp2m-34w__gfdlflora0',
'nmme0-tmp2m-34w__gfdlflorb0',
'nmme0-tmp2m-34w__gfdl0',
'nmme0-tmp2m-34w__nasa0']

g_2 = ['nmme-tmp2m-56w__cancm3',
'nmme-tmp2m-56w__cancm4',
'nmme-tmp2m-56w__ccsm4',
'nmme-tmp2m-56w__cfsv2',
'nmme-tmp2m-56w__gfdl',
'nmme-tmp2m-56w__gfdlflora',
'nmme-tmp2m-56w__gfdlflorb',
'nmme-tmp2m-56w__nasa']

g_3 = ['nmme-prate-34w__cancm3',
'nmme-prate-34w__cancm4',
'nmme-prate-34w__ccsm4',
'nmme-prate-34w__cfsv2',
'nmme-prate-34w__gfdl',
'nmme-prate-34w__gfdlflora',
'nmme-prate-34w__gfdlflorb',
'nmme-prate-34w__nasa']

g_4 = [ 'nmme0-prate-56w__cancm30',
'nmme0-prate-56w__cancm40',
'nmme0-prate-56w__ccsm40',
'nmme0-prate-56w__cfsv20',
'nmme0-prate-56w__gfdlflora0',
'nmme0-prate-56w__gfdlflorb0',
'nmme0-prate-56w__gfdl0',
'nmme0-prate-56w__nasa0']

g_5 = ['nmme0-prate-34w__cancm30',
'nmme0-prate-34w__cancm40',
'nmme0-prate-34w__ccsm40',
'nmme0-prate-34w__cfsv20',
'nmme0-prate-34w__gfdlflora0',
'nmme0-prate-34w__gfdlflorb0',
'nmme0-prate-34w__gfdl0',
'nmme0-prate-34w__nasa0']

g_6 = ['nmme-prate-56w__cancm3',
'nmme-prate-56w__cancm4',
'nmme-prate-56w__ccsm4',
'nmme-prate-56w__cfsv2',
'nmme-prate-56w__gfdl',
'nmme-prate-56w__gfdlflora',
'nmme-prate-56w__gfdlflorb',
'nmme-prate-56w__nasa']

g_7 = ['nmme-tmp2m-34w__cancm3',
'nmme-tmp2m-34w__cancm4',
'nmme-tmp2m-34w__ccsm4',
'nmme-tmp2m-34w__cfsv2',
'nmme-tmp2m-34w__gfdl',
'nmme-tmp2m-34w__gfdlflora',
'nmme-tmp2m-34w__gfdlflorb',
'nmme-tmp2m-34w__nasa']

In [7]:
gs = [g_1, g_2, g_3, g_4, g_5, g_6, g_7]

zip_cols = zip(col_na, gs, g_means)
for c, g, m in (zip_cols):
    X[c] = (X[m]*9) - X[g].sum(1)
X['ccsm30'] = 9*X['nmme0mean'] - (X['cancm30']+X['cancm40']+X['ccsm40']+X['cfsv20']+X['gfdlflora0']+X['gfdlflorb0']+X['gfdl0']+X['nasa0'])

filter_na_cols(X)

'Clean dataset'

In [8]:
# add features
X['YearsSinceRecordings'] = datetime.datetime.now().year - X.year
X_test['YearsSinceRecordings'] = datetime.datetime.now().year - X_test.year

In [9]:
def identify_correlated(df, threshold):
    corr_matrix = df.corr().abs()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    reduced_corr_matrix = corr_matrix.mask(mask)
    features_to_drop = [c for c in reduced_corr_matrix.columns if any(reduced_corr_matrix[c] > threshold)]
    return features_to_drop

In [10]:
features_to_drop = identify_correlated(X, .80)
X_reduced = pd.DataFrame(X.drop(features_to_drop, axis=1))
X_test_reduced = pd.DataFrame(X_test.drop(features_to_drop, axis=1))

In [11]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    #os.environ['CUDA_LAUNCH_BLOCKING'] = 1
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True



In [12]:
categorical_features_indices = np.where((X_reduced.dtypes != np.float64))

list_cols= list(X_reduced.columns)
cat_cols = []
for i in categorical_features_indices[0]:
    cat_cols.append(list_cols[i])

cont_cols = list(X_reduced.columns).copy()
for col in cat_cols:
    cont_cols.remove(col) 

train_cols = list(X_reduced.columns)

In [13]:
cat_cols

['climateregions__climateregion',
 'loc_group',
 'day_of_year',
 'YearsSinceRecordings']

In [14]:
train_df = pd.concat([X_reduced,y],axis=1)
test_df = X_test_reduced

In [15]:
# Extract the index of train and val 
X_train, X_val, y_train, y_val = train_test_split(X_reduced, y, test_size=0.33, random_state=42)

train_index = X_train.index
val_index = X_val.index

## Handling Continuos Features

In [16]:
# Robust Scaling

scaler = RobustScaler(quantile_range=(10.0, 90.0))
scaler.fit(pd.concat([train_df[cont_cols], test_df[cont_cols]]))

train_df[cont_cols] = scaler.transform(train_df[cont_cols].values)
test_df[cont_cols] = scaler.transform(test_df[cont_cols].values)


In [17]:
# train numerical tensor
contTensor = np.stack([train_df[col] for col in cont_cols],axis=1)
contTensor = torch.tensor(contTensor,dtype=torch.float)

# test numerical tensor
testcontTensor = np.stack([test_df[col] for col in cont_cols],axis=1)
testcontTensor = torch.tensor(testcontTensor,dtype=torch.float)

## Handling Categorical Features
- Les embeddings en bas Just to see how it works (it will be handled in the model)

In [18]:
uniques = {}

for col in cat_cols:
    le = LabelEncoder()
    le.fit(pd.concat([train_df[col], test_df[col]]))
    train_df[col] = le.transform(train_df[col].values)
    test_df[col] = le.transform(test_df[col].values)
    uniques[col] = len(pd.concat([train_df[col], test_df[col]], axis=0).unique())

In [19]:
catTensor = np.stack([train_df[col] for col in cat_cols],1)
catTensor = torch.tensor(catTensor,dtype=torch.int64)
catTensor

tensor([[  0,   0, 243,   3],
        [  0,   0, 244,   3],
        [  0,   0, 245,   3],
        ...,
        [  9, 513, 241,   1],
        [  9, 513, 242,   1],
        [  9, 513, 243,   1]])

In [20]:
testcatTensor = np.stack([test_df[col] for col in cat_cols],1)
testcatTensor = torch.tensor(testcatTensor,dtype=torch.int64)
testcatTensor

tensor([[  0,   0, 304,   0],
        [  0,   0, 305,   0],
        [  0,   0, 306,   0],
        ...,
        [  9, 513, 362,   0],
        [  9, 513, 363,   0],
        [  9, 513, 364,   0]])

-  **embeddings dims**

In [21]:
cat_dims = list(uniques.values())
cat_dims

[15, 514, 365, 4]

In [22]:
embedding_dims =[(15,7),(514,180),(365,20),(4,3)]

## Building the Model

In [23]:
seed_everything(seed=42)
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)


class ModelWIDS2023(nn.Module):
    def __init__(self, embedding_dim, n_cont, out_sz, layers,device=device, drop=0.5):
        super().__init__()
        self.device = device
        #Embedding layers
        self.embed_repr = nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dims])
        self.embed_dropout = nn.Dropout(drop)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        
        layerlist = []
        n_emb = sum((val[1] for val in embedding_dim))
        n_in = n_cont + n_emb
        
        # FastiAI approach: apply BN and dropout before relu
        for layer in layers:
            layerlist.append(nn.Linear(n_in,layer))
            layerlist.append(nn.Dropout(drop))
            #layerlist.append(nn.BatchNorm1d(layer))
            layerlist.append(nn.ReLU(inplace=True))
            n_in = layer
        layerlist.append(nn.Linear(layers[-1],out_sz))
        
        self.layers = nn.Sequential(*layerlist)
        # Initialize weights in the two linear layers
        for lin_layer in self.layers:
            lin_layer.apply(init_weights)
            #if type(lin_layer) == nn.Linear:
                #nn.init.kaiming_normal_(lin_layer.weight.data)
        
    def forward(self, cat,cont):
        cont = cont.to(self.device)
        cat= cat.to(self.device)
        embeddings = []
        for i,e in enumerate(self.embed_repr):
            embeddings.append(e(cat[:,i]))
        x = torch.cat(embeddings,1)
        x = self.embed_dropout(x)#: donne mieux sans ça
        #x_cont = self.bn_cont(cont)
        x = torch.cat([x,cont],1)
        x = self.layers(x)
        return x

In [25]:
model = ModelWIDS2023(embedding_dims, len(cont_cols), 1, [512,256], drop=0).to(device)

In [26]:
model

ModelWIDS2023(
  (embed_repr): ModuleList(
    (0): Embedding(15, 7)
    (1): Embedding(514, 180)
    (2): Embedding(365, 20)
    (3): Embedding(4, 3)
  )
  (embed_dropout): Dropout(p=0, inplace=False)
  (bn_cont): BatchNorm1d(142, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=352, out_features=512, bias=True)
    (1): Dropout(p=0, inplace=False)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): Dropout(p=0, inplace=False)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [27]:
train_cat = catTensor[train_index]
val_cat = catTensor[val_index]

train_cont = contTensor[train_index]
val_cont = contTensor[val_index]

y= torch.tensor(y,dtype=torch.float).reshape(-1,1)
y_train = y[train_index]
y_val = y[val_index]

In [28]:
epochs = 5000

loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-5)


In [37]:
losses = []
for i in range(epochs):
    i += 1
    y_pred = model.forward(train_cat,train_cont)
    loss = torch.sqrt(loss_function(y_pred,y_train.to(device)))
    losses.append(loss)
    if i%50 == 0:
        print(f"Epoch {i} : {loss}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #scheduler.step()

Epoch 50 : 2.4576966762542725
Epoch 100 : 1.0172197818756104
Epoch 150 : 0.5955129265785217
Epoch 200 : 0.4326685965061188
Epoch 250 : 0.33958926796913147
Epoch 300 : 0.34417739510536194
Epoch 350 : 0.2858445346355438
Epoch 400 : 0.26356828212738037
Epoch 450 : 0.2457670420408249
Epoch 500 : 0.22971747815608978
Epoch 550 : 0.22052985429763794
Epoch 600 : 0.21152520179748535
Epoch 650 : 0.20356982946395874
Epoch 700 : 0.19699883460998535
Epoch 750 : 0.19086650013923645
Epoch 800 : 0.18603746592998505
Epoch 850 : 0.18127407133579254
Epoch 900 : 0.1775096207857132
Epoch 950 : 0.17375358939170837
Epoch 1000 : 0.17074063420295715
Epoch 1050 : 0.16767717897891998
Epoch 1100 : 0.1652105748653412
Epoch 1150 : 0.16234055161476135
Epoch 1200 : 0.16008633375167847
Epoch 1250 : 0.1579250544309616
Epoch 1300 : 0.15586405992507935
Epoch 1350 : 0.15403179824352264
Epoch 1400 : 0.15224039554595947
Epoch 1450 : 0.1504911482334137
Epoch 1500 : 0.14901253581047058
Epoch 1550 : 0.1473817229270935
Epoch 16

In [39]:
with torch.no_grad():
    y_pred=model(val_cat,val_cont)
    loss=torch.sqrt(loss_function(y_pred,y_val.to(device)))
print('RMSE: {}'.format(loss))

RMSE: 0.16021375358104706


In [40]:
test_cat = testcatTensor
test_cont = testcontTensor

with torch.no_grad():
    preds_test=model(test_cat,test_cont)

preds_test = preds_test.detach().cpu().numpy().flatten()
preds_test

array([29.490929 , 29.633265 , 29.680397 , ...,  7.216925 ,  7.8647537,
        8.159615 ], dtype=float32)

### Save the model

In [41]:
model_root = f"../Pytorch&Keras/modelpytorch_3_nn.pt"
torch.save(model, model_root)

In [29]:
# jsut for verification
model_root = f"../Pytorch&Keras/models_pytorch/modelpytorch_3_nn.pt"

model_final = torch.load(model_root)
with torch.no_grad():
    test_cat = testcatTensor
    test_cont = testcontTensor
    preds_test_verif=model_final(test_cat,test_cont)
pred_test = preds_test_verif.detach().cpu().numpy().flatten()
pred_test

array([29.490929 , 29.633265 , 29.680397 , ...,  7.216925 ,  7.8647537,
        8.159615 ], dtype=float32)

In [30]:
submit[target] = pred_test
submit.to_csv('../pytorch_model_1.csv',index=False)

In [44]:
# The last submission (that gave us 0.627)
best627 = pd.read_csv('../fast_ai_regression/submissions_with_fast_ai/(0.627)sub631_0.9_fastaufeat0.1.csv')[target]


## PsuedoLabeling

In [47]:
model_best = torch.load('../Pytorch&Keras/modelpytorch_3_nn.pt')

In [48]:
with torch.no_grad():
    preds_test_verif=model_best(test_cat,test_cont)
pred_test = preds_test_verif.detach().cpu().numpy().flatten()
pred_test

array([29.490929 , 29.633265 , 29.680397 , ...,  7.216925 ,  7.8647537,
        8.159615 ], dtype=float32)

- **Concatenate all**

In [49]:
y_test = torch.tensor(preds_test,dtype=torch.float).reshape(-1,1)

In [50]:
X_tot_cat = torch.cat([train_cat,val_cat,test_cat],0)
X_tot_cont = torch.cat([train_cont, val_cont,test_cont],0)
y_tot = torch.cat([y_train, y_val,y_test],0)

In [51]:
epochs = 5000

loss_function = nn.MSELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr =0.01)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-5)

In [52]:
losses = []
for i in range(epochs):
    i += 1
    y_pred = model.forward(X_tot_cat,X_tot_cont)
    loss = torch.sqrt(loss_function(y_pred,y_tot.to(device)))
    losses.append(loss)
    if i%50 == 0:
        print(f"Epoch {i} : {loss}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #scheduler.step()

Epoch 50 : 0.13922029733657837
Epoch 100 : 0.11930844187736511
Epoch 150 : 0.11668518930673599
Epoch 200 : 0.1156155988574028
Epoch 250 : 0.11458556354045868
Epoch 300 : 0.1154228076338768
Epoch 350 : 0.11320791393518448
Epoch 400 : 0.11238934844732285
Epoch 450 : 0.11118602752685547
Epoch 500 : 0.11137665808200836
Epoch 550 : 0.11074534058570862
Epoch 600 : 0.11011864244937897
Epoch 650 : 0.10966353118419647
Epoch 700 : 0.10936149954795837
Epoch 750 : 0.10916829854249954
Epoch 800 : 0.10849349945783615
Epoch 850 : 0.10806155204772949
Epoch 900 : 0.10755898803472519
Epoch 950 : 0.10746560245752335
Epoch 1000 : 0.10655444115400314
Epoch 1050 : 0.10678943246603012
Epoch 1100 : 0.10588598996400833
Epoch 1150 : 0.10601924359798431
Epoch 1200 : 0.10552255809307098
Epoch 1250 : 0.10542131215333939
Epoch 1300 : 0.10500877350568771
Epoch 1350 : 0.10489461570978165
Epoch 1400 : 0.10443870723247528
Epoch 1450 : 0.10396043956279755
Epoch 1500 : 0.10385225713253021
Epoch 1550 : 0.10387058556079865

In [57]:
model_root = f"../Pytorch&Keras/modelpytorch_3_psuedo_nn.pt"
torch.save(model, model_root)

In [58]:
with torch.no_grad():
    preds_test_pseudo = model(test_cat,test_cont)
preds_test_pseudo = preds_test_pseudo.detach().cpu().numpy().flatten()
preds_test_pseudo

array([29.37088 , 29.50084 , 29.532034, ...,  7.125566,  7.762993,
        8.086815], dtype=float32)

In [59]:
0.95*best627 +0.05*preds_test_pseudo # gives 0.622

0        29.262159
1        29.270729
2        29.290475
3        29.348092
4        29.413255
           ...    
31349     6.667905
31350     6.859207
31351     6.239443
31352     7.044306
31353     7.555243
Name: contest-tmp2m-14d__tmp2m, Length: 31354, dtype: float64

In [32]:
#just verification
model_root = '../Pytorch&Keras/models_pytorch/modelpytorch_3_psuedo_nn.pt'

model_final = torch.load(model_root)
with torch.no_grad():
    test_cat = testcatTensor
    test_cont = testcontTensor
    preds_test_verif=model_final(test_cat,test_cont)
pred_test = preds_test_verif.detach().cpu().numpy().flatten()
pred_test

array([29.37088 , 29.50084 , 29.532034, ...,  7.125566,  7.762993,
        8.086815], dtype=float32)

In [33]:
submit[target] = pred_test
submit.to_csv('../pytorch_model_2_pseudo.csv',index=False)

In [60]:
submit[target] = 0.95*best627 +0.05*preds_test_pseudo # gives 0.622
submit.to_csv('../Pytorch&Keras/best627_0.95_pytorch_mod3pseudo_0.05.csv',index=False)